In [6]:
###
filepath = '1999_referenda/results_by_polling_place/v4_3ADEL.txt' 

f = open(filepath, 'r')
data = f.read()

print(data)

Referendum: 6 November 1999									
			
Referendum Results										
		
By Division by Polling Place									
			

"A proposed law for the Alteration of the Constitution, entitled"			
									
   the Constitutional Alteration (Establishment of Republic) 1999			
									
												
												
South Australia											
	
Adelaide (86272)											
	
												
		Yes		No		Formal		Informal		Total	
	
Polling Place		Votes	%	Votes	%	Votes	%	Votes	%	Votes	%
	
Adelaide (Adelaide)	Yes	282	61.44	177	38.56	459	98.71	6	1.29
	465	0.57	
Adelaide East	Yes	465	67.00	229	33.00	694	99.14	6	0.86	700
	0.86	
Adelaide Hospital	Yes	187	61.72	116	38.28	303	98.06	6	1.94	309
	0.38	
Adelaide South	Yes	1056	64.67	577	35.33	1633	99.03	16	0.97	1649
	2.02
Adelaide West	Yes	205	64.87	111	35.13	316	98.75	4	1.25	320
	0.39
Black Forest East	Yes	1206	60.09	801	39.91	2007	98.58	29	1.42	2036
	2.50
Blair Athol North	No	628	41.02	903	58.98	1531	99.22	12	0.78	1543
	1.89
Broadview	No	971	47.93	1055	52.07	2026	99.46	1

In [24]:
import linecache

## in these text files, the 14th line is the state, the 16th is the electorate
state = linecache.getline(filepath, 14).strip()
electorate = linecache.getline(filepath, 16).strip()

# electorate contains name and enrollment in the form "Adelaide (86272)"
needle = electorate.find('(')
enrolment = electorate[needle:].strip() # take right of needle
enrolment = enrolment[1:-1] # remove first and last chars, ie, remove brackets

electorate = electorate[:needle-1].strip() # take everything from the start to one place before the bracket
#print(state + electorate + enrolment)

South AustraliaAdelaide86272


In [39]:
import pandas as pd

filepath = '1999_referenda/results_by_polling_place/v4_3ADEL.XLS'
columnheaders = ['polling_place_raw',
'yes_or_no',
'yes_n',
'yes_p',
'no_n',
'no_p',
'formal_n',
'formal_p',
'informal_n',
'informal_p',
'total_n',
'total_p']

df = pd.read_excel(
    filepath,
    skiprows = 13,
    names=columnheaders
)

print(df)

     polling_place_raw yes_or_no  yes_n  yes_p   no_n   no_p  formal_n  \
0        Arthurs Creek        No    148  45.82    175  54.18       323   
1               Avenel        No    206  35.15    380  64.85       586   
2         Badger Creek        No    518  41.05    744  58.95      1262   
3            Beveridge        No    108  43.37    141  56.63       249   
4          Bonnie Doon        No    110  33.13    222  66.87       332   
5            Broadford        No    723  34.91   1348  65.09      2071   
6               Buxton        No     74  32.89    151  67.11       225   
7      Christmas Hills       Yes     35  72.92     13  27.08        48   
8           Clonbinane        No    104  41.94    144  58.06       248   
9          Craigieburn        No   2161  44.69   2674  55.31      4835   
10   Craigieburn South        No   1011  45.54   1209  54.46      2220   
11       Diamond Creek       Yes   1547  56.42   1195  43.58      2742   
12  Diamond Creek East       Yes   137